In [ ]:
# https://ibkrcampus.com/campus/ibkr-api-page/twsapi-doc/#api-introduction
#pip install requests
# https://stackoverflow.com/questions/8548030/why-does-pip-install-inside-python-raise-a-syntaxerror

In [1]:
from ib_insync import *
import requests
import pandas as pd
from bs4 import BeautifulSoup

util.startLoop()  # only use in interactive environments (i.e. Jupyter Notebooks)
# https://algotrading101.com/learn/ib_insync-interactive-brokers-api-guide/

In [2]:
#WORKING
ib = IB()
ib.connect(host='127.0.0.1', port=7496, clientId=0)
# https://www.elitetrader.com/et/threads/cant-connect-to-api-interactive-brokers.361205/
# needed to change socket port to 7496 and uncheck the Read-Only API box

<IB connected to 127.0.0.1:7496 clientId=0>

In [3]:
ib.positions()
pending_trades = ib.reqOpenOrders() # requires that I use ClientId=0 rather than ClientId=1
# https://www.reddit.com/r/interactivebrokers/comments/119begz/cant_see_open_orders_via_api/

[Trade(contract=Stock(conId=641842650, symbol='DRUG', right='?', exchange='SMART', currency='USD', localSymbol='DRUG', tradingClass='SCM'), order=Order(orderId=-2, permId=887072239, action='BUY', totalQuantity=1.0, orderType='LMT', lmtPrice=0.0155, auxPrice=0.0, tif='GTC', ocaType=3, displaySize=2147483647, rule80A='0', openClose='', volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U12848716', clearingIntent='IB', adjustedOrderType='None', cashQty=0.0, dontUseAutoPriceForHedge=True), orderStatus=OrderStatus(orderId=-2, status='PreSubmitted', filled=0.0, remaining=1.0, avgFillPrice=0.0, permId=887072239, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2024, 11, 12, 3, 29, 27, 320746, tzinfo=datetime.timezone.utc), status='PreSubmitted', message='', errorCode=0)], advancedError=''),
 Trade(contract=Stock(conId=528660539, symbol='CERO', right='?', exchange='SMART', currency='USD', lo

In [ ]:
# Sample data structure based on the provided input
trades = [
    {
        "contract": {"conId": 641842650, "symbol": "DRUG", "right": "?", "exchange": "SMART", "currency": "USD", "localSymbol": "DRUG", "tradingClass": "SCM"},
        "order": {"orderId": -2, "permId": 887072239, "action": "BUY", "totalQuantity": 1.0, "orderType": "LMT", "lmtPrice": 0.0155},
        "orderStatus": {"orderId": -2, "status": "PreSubmitted"}
    }
]

# Extracting the "symbol" value from each trade
symbols = [trade["contract"]["symbol"] for trade in trades]

# Print the list of symbols
print(symbols)

In [ ]:
# https://github.com/Siloaman/ib_async/blob/main/notebooks/bar_data.ipynb
# To get the earliest date of available bar data the "head timestamp" can be requested:
contract = Stock('TSLA', 'SMART', 'USD')
ib.reqHeadTimeStamp(contract, whatToShow='TRADES', useRTH=True)

In [ ]:
# Legal ones are: 
# 1 secs, 5 secs, 10 secs, 15 secs, 30 secs, 
# 1 min, 2 mins, 3 mins, 5 mins, 10 mins, 15 mins, 20 mins, 30 mins, 
# 1 hour, 2 hours, 3 hours, 4 hours, 8 hours, 
# 1 day, 1W, 1M
# To request hourly data of the last 1 day per 5 minutes:
bars = ib.reqHistoricalData(
        contract,
        endDateTime='',
        durationStr='1 D',
        barSizeSetting='5 mins',
        whatToShow='TRADES',
        useRTH=True,
        formatDate=1)

bars[0]

In [ ]:
df = util.df(bars)
display(df.tail(8))

In [ ]:
%matplotlib inline

In [ ]:
# There is also a utility function to plot bars as a candlestick plot. 
# It can accept either a DataFrame or a list of bars.
# Here it will print the last 10 bars:
util.barplot(bars[-10:], title=contract.symbol);

In [ ]:
# A new feature of the API is to get live updates for historical bars.
# This is done by setting endDateTime to an empty string and the keepUpToDate parameter to True.
bars = ib.reqHistoricalData(
        contract,
        endDateTime='',
        durationStr='480 S',          # 5 mins x 8 bars (60 x 8)
        barSizeSetting='5 mins',
        whatToShow='MIDPOINT',
        useRTH=True,
        formatDate=1,
        keepUpToDate=True)

In [ ]:
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

def onBarUpdate(bars, hasNewBar):
    plt.close()
    plot = util.barplot(bars)
    clear_output(wait=True)
    display(plot)

bars.updateEvent += onBarUpdate

ib.sleep(10)
ib.cancelHistoricalData(bars)
 # barUpdateEvent (bars: BarDataList, hasNewBar: bool): Emits the bar list that has been updated in real time.
 # If a new bar has been added then hasNewBar is True, when the last bar has changed it is False.


In [ ]:
# WORKING
acct_values = ib.accountValues()
#cash_balance = acct_values

#    [
#    AccountValue(account='U12848716', tag='AccountCode', value='U12848716', currency='', modelCode=''),
#    AccountValue(account='U12848716', tag='CashBalance', value='517.53', currency='USD', modelCode=''),
#    AccountValue(account='U12848716', tag='GrossPositionValue', value='1.58', currency='CAD', modelCode=''),
#    AccountValue(account='U12848716', tag='NetLiquidation', value='723.69', currency='CAD', modelCode=''),
#    AccountValue(account='U12848716', tag='TotalCashBalance', value='2.24', currency='CAD', modelCode=''),
#    AccountValue(account='U12848716', tag='TotalCashBalance', value='517.53', currency='USD', modelCode=''),
#    ]

Get Overall USD Cash Balance

In [ ]:
# WORKING
def get_cash_balance_usd(acct_values):
    for item in acct_values:
        # AccountValue(account='U12848716', tag='CashBalance', value='517.53', currency='USD', modelCode=''),
        if item[1] == 'CashBalance' and item[3] == 'USD':
            return float(item[2])  # Convert value to float for numerical use

# Get the CashBalance in USD
cash_balance_usd = get_cash_balance_usd(acct_values)
cash_balance_usd

# Retrieving Historical Data

In [ ]:
# for this to work, Market Data Subscriber Status needs to be updated for a subscription

nflx_contract = Stock('mlgo', 'SMART', 'USD')     # netflix ticker, 'smart' search for routing exchange, currency of ticker

ib.qualifyContracts(nflx_contract)

data = ib.reqMktData(nflx_contract)

In [ ]:
historical_data_nflx = ib.reqHistoricalData(
    nflx_contract, 
    '', 
    barSizeSetting='5 mins', 
    durationStr='1 D', 
    whatToShow='MIDPOINT', 
    useRTH=True
    )

historical_data_nflx[-6]      #  access the most recent candle, just like any other Python list, by using a negative index.
# https://www.interactivebrokers.com/campus/ibkr-api-page/market-data-subscriptions/
#historical_data_nflx[-1].open

In [ ]:
ib.run()